In [ ]:
!pip install mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 12.7 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from typing import List, Literal, Optional, Dict, Any
from pydantic import BaseModel
from mistralai import Mistral


API_KEY = userdata.get("MISTRAL_API_KEY")
client = Mistral(api_key=API_KEY)
model = "mistral-medium-2505"

Парсинг с вики и сохранение дока с ПДД

In [1]:
import requests
from bs4 import BeautifulSoup

def parse_wikisource_precise(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        content = soup.find('div', {'class': 'mw-parser-output'})

        if not content:
            return None

        for element in content.find_all(['div', 'table', 'span']):
            classes = element.get('class', [])
            if any(cls in classes for cls in ['navbox', 'toc', 'infobox', 'reference']):
                element.decompose()


        text_content = []
        for element in content.find_all(['p', 'h2', 'h3', 'h4']):
            text = element.get_text().strip()
            text = ' '.join(text.split())
            if text and len(text) > 20:  # Минимальная длина текста
                text_content.append(text)

        return text_content

    except Exception as e:
        print(f"Ошибка: {e}")
        return None

def save_text_to_file(text_elements, filename):
    try:
        with open(filename, 'w', encoding='utf-8') as file:
            for i, text in enumerate(text_elements, 1):
                file.write(f"{text}\n\n")
        print(f"Текст успешно сохранен в файл: {filename}")
    except Exception as e:
        print(f"Ошибка при сохранении в файл: {e}")


url = "https://ru.wikisource.org/wiki/%D0%9F%D1%80%D0%B0%D0%B2%D0%B8%D0%BB%D0%B0_%D0%B4%D0%BE%D1%80%D0%BE%D0%B6%D0%BD%D0%BE%D0%B3%D0%BE_%D0%B4%D0%B2%D0%B8%D0%B6%D0%B5%D0%BD%D0%B8%D1%8F_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%BE%D0%B9_%D0%A4%D0%B5%D0%B4%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D0%B8"

text = parse_wikisource_precise(url)
if text:
    filename = "правила_дорожного_движения.txt"
    save_text_to_file(text, filename)

    for i, paragraph in enumerate(text[:1000], 1):
        print(f"{i}. {paragraph}")
        print()

Текст успешно сохранен в файл: правила_дорожного_движения.txt
1. 1.1. Настоящие Правила дорожного движения* устанавливают единый порядок дорожного движения на всей территории Российской Федерации. Другие нормативные акты, касающиеся дорожного движения, должны основываться на требованиях Правил и не противоречить им.

2. 1.2. В Правилах используются следующие основные понятия и термины:

3. «Автомагистраль» — дорога, обозначенная знаком 5.1** и имеющая для каждого направления движения проезжие части, отделенные друг от друга разделительной полосой (а при ее отсутствии — дорожным ограждением), без пересечений в одном уровне с другими дорогами, железнодорожными или трамвайными путями, пешеходными или велосипедными дорожками.

4. «Автопоезд» — механическое транспортное средство, сцепленное с прицепом (прицепами).

5. «Велосипед» — транспортное средство, кроме инвалидных колясок, которое имеет по крайней мере два колеса и приводится в движение как правило мускульной энергией лиц, находящихс

In [2]:
!pip install faiss-cpu sentence-transformers numpy scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 91.5 MB/s eta 0:00:00


Вариант faiss по итоговому тексту с дополнительным разбитием на подпункты перед итоговой векторизацией
---

In [3]:
import re
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize
import pickle
import os

class PDDFAISSCreator:
    def __init__(self, model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'):
        """
        Инициализация класса для создания векторного хранилища ПДД

        Args:
            model_name: название модели для эмбеддингов
        """
        self.model = SentenceTransformer(model_name)
        self.index = None
        self.documents = []
        self.sections = []

    def load_and_parse_pdd_file(self, file_path):
        """
        Загрузка и парсинг файла с ПДД

        Args:
            file_path: путь к файлу с правилами дорожного движения
        """
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

        sections = re.split(r'(\d+\.\d+\.)', content)

        current_section = ""
        section_number = ""

        for i, section in enumerate(sections):
            if re.match(r'\d+\.\d+\.', section):
                if current_section and section_number:
                    self.sections.append({
                        'number': section_number.strip('.'),
                        'content': current_section.strip(),
                        'full_text': f"{section_number} {current_section}".strip()
                    })
                    self.documents.append(f"{section_number} {current_section}".strip())

                section_number = section
                current_section = ""
            else:
                current_section += section

        if current_section and section_number:
            self.sections.append({
                'number': section_number.strip('.'),
                'content': current_section.strip(),
                'full_text': f"{section_number} {current_section}".strip()
            })
            self.documents.append(f"{section_number} {current_section}".strip())

        print(f"Загружено разделов: {len(self.documents)}")

    def create_embeddings(self):
        """
        Создание эмбеддингов для всех документов
        """
        if not self.documents:
            raise ValueError("Сначала загрузите документы!")

        print("Создание эмбеддингов...")
        embeddings = self.model.encode(self.documents, show_progress_bar=True)

        # Нормализуем векторы для косинусного сходства
        embeddings = normalize(embeddings, norm='l2', axis=1)

        return embeddings.astype('float32')

    def build_faiss_index(self, embeddings, index_type='IVF'):
        """
        Построение FAISS индекса

        Args:
            embeddings: матрица эмбеддингов
            index_type: тип индекса ('IVF', 'Flat', 'HNSW')
        """
        dimension = embeddings.shape[1]

        if index_type == 'Flat':
            self.index = faiss.IndexFlatIP(dimension)
        elif index_type == 'IVF':
            nlist = min(100, len(embeddings) // 39)
            quantizer = faiss.IndexFlatIP(dimension)
            self.index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_INNER_PRODUCT)


            if len(embeddings) >= nlist:
                self.index.train(embeddings)
        elif index_type == 'HNSW':
            self.index = faiss.IndexHNSWFlat(dimension, 32, faiss.METRIC_INNER_PRODUCT)

        # Добавляем векторы в индекс
        if index_type == 'IVF' and len(embeddings) >= nlist:
            self.index.add(embeddings)
        else:
            self.index.add(embeddings)

        print(f"Индекс создан. Размер: {self.index.ntotal} векторов")

    def save_index(self, save_path='pdd_faiss_index'):
        """
        Сохранение индекса и метаданных

        Args:
            save_path: путь для сохранения
        """
        if not self.index:
            raise ValueError("Сначала создайте индекс!")

        os.makedirs(save_path, exist_ok=True)

        faiss.write_index(self.index, f'{save_path}/pdd.index')

        metadata = {
            'documents': self.documents,
            'sections': self.sections,
            'model_name': self.model.get_sentence_embedding_dimension()
        }

        with open(f'{save_path}/metadata.pkl', 'wb') as f:
            pickle.dump(metadata, f)

        print(f"Индекс и метаданные сохранены в {save_path}/")

    def load_index(self, load_path='pdd_faiss_index'):
        """
        Загрузка индекса и метаданных

        Args:
            load_path: путь для загрузки
        """

        self.index = faiss.read_index(f'{load_path}/pdd.index')

        with open(f'{load_path}/metadata.pkl', 'rb') as f:
            metadata = pickle.load(f)

        self.documents = metadata['documents']
        self.sections = metadata['sections']

        print(f"Индекс загружен. Размер: {self.index.ntotal} векторов")

    def search(self, query, k=5):
        """
        Поиск похожих документов

        Args:
            query: поисковый запрос
            k: количество возвращаемых результатов

        Returns:
            list: список кортежей (индекс, схожесть, документ)
        """
        if not self.index:
            raise ValueError("Сначала загрузите или создайте индекс!")

        query_embedding = self.model.encode([query])
        query_embedding = normalize(query_embedding, norm='l2', axis=1).astype('float32')

        # Поиск в FAISS
        similarities, indices = self.index.search(query_embedding, k)

        results = []
        for i, (similarity, idx) in enumerate(zip(similarities[0], indices[0])):
            if idx < len(self.documents):  # Проверка на валидность индекса
                results.append({
                    'rank': i + 1,
                    'similarity': float(similarity),
                    'section_number': self.sections[idx]['number'],
                    'content': self.sections[idx]['content'],
                    'full_text': self.documents[idx]
                })

        return results

def create_pdd_vector_store(file_path, save_path='pdd_faiss_index', index_type='IVF'):
    """
    Функция для создания векторного хранилища ПДД

    Args:
        file_path: путь к файлу с ПДД
        save_path: путь для сохранения индекса
        index_type: тип FAISS индекса
    """
    pdd_creator = PDDFAISSCreator()

    print("Загрузка и парсинг файла ПДД...")
    pdd_creator.load_and_parse_pdd_file(file_path)

    embeddings = pdd_creator.create_embeddings()

    print("Построение FAISS индекса...")
    pdd_creator.build_faiss_index(embeddings, index_type)

    pdd_creator.save_index(save_path)

    print("Векторное хранилище успешно создано!")

    print("\nТестовый поиск:")
    results = pdd_creator.search("скорость движения в населенном пункте", k=3)
    for result in results:
        print(f"\n{result['rank']}. Схожесть: {result['similarity']:.3f}")
        print(f"Раздел: {result['section_number']}")
        print(f"Содержание: {result['content'][:200]}...")

    return pdd_creator


if __name__ == "__main__":
    file_path = "/content/правила_дорожного_движения.txt"
    pdd_vector_store = create_pdd_vector_store(file_path)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Загрузка и парсинг файла ПДД...
Загружено разделов: 259
Создание эмбеддингов...


Batches:   0%|          | 0/9 [00:00<?, ?it/s]

Построение FAISS индекса...
Индекс создан. Размер: 259 векторов
Индекс и метаданные сохранены в pdd_faiss_index/
Векторное хранилище успешно создано!

Тестовый поиск:

1. Схожесть: 0.787
Раздел: 10.3
Содержание: Вне населенных пунктов разрешается движение:

Примечание. По решению собственников или владельцев автомобильных дорог может разрешаться повышение скорости на участках дорог для отдельных видов транспо...

2. Схожесть: 0.739
Раздел: 10.2
Содержание: В населенных пунктах разрешается движение транспортных средств со скоростью не более 60 км/ч, а в жилых зонах и на дворовых территориях не более 20 км/ч.

(Примечание. По решению органов исполнительно...

3. Схожесть: 0.702
Раздел: 5.3
Содержание: 17. Движение в жилых зонах...


In [4]:
result = pdd_vector_store.search('движение на перекрестке', k=5)

In [5]:
for i, elem in enumerate(result):
  print('='*60 + f' {i} ' + '='*60)
  print(elem)

============================================================ 0 ============================================================
{'rank': 1, 'similarity': 0.7499738931655884, 'section_number': '1.14', 'content': '2, обозначающей такой пешеходный переход.\n\nПри отсутствии в зоне видимости перехода или перекрестка разрешается переходить дорогу под прямым углом к краю проезжей части на участках без разделительной полосы и ограждений там, где она хорошо просматривается в обе стороны.', 'full_text': '1.14. 2, обозначающей такой пешеходный переход.\n\nПри отсутствии в зоне видимости перехода или перекрестка разрешается переходить дорогу под прямым углом к краю проезжей части на участках без разделительной полосы и ограждений там, где она хорошо просматривается в обе стороны.'}
============================================================ 1 ============================================================
{'rank': 2, 'similarity': 0.7254129648208618, 'section_number': '24.4', 'content': 'На нерегулируе

In [1]:
!pip install langchain langchain-community faiss-cpu sentence-transformers langchain-huggingface

Варик в тупую как из ноутбука с одной из практик нашего курса по LLM
---

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

with open('/content/правила_дорожного_движения.txt', 'r') as file:
  documents = file.readlines()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
texts = []
for doc in documents:
    texts.extend(text_splitter.split_text(doc))


embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
faiss_store = FAISS.from_texts(texts, embeddings)
faiss_retriever = faiss_store.as_retriever()



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
query = "обгон в неположенном месте"
results = faiss_retriever.invoke(query)

print(f"Результаты для запроса: '{query}'\n")
for i, doc in enumerate(results):
    print(f"{i+1}. {doc.page_content}")
    print("-" * 50)

Результаты для запроса: 'обгон в неположенном месте'

1. Нерегулируемые перекрестки
--------------------------------------------------
2. 11.4. Обгон запрещен:
--------------------------------------------------
3. перечень дорог, на которых запрещается учебная езда;
--------------------------------------------------
4. Кроме того, запрещается:
--------------------------------------------------
